# 📡 Sensor Band Fairness Analysis Framework

This notebook demonstrates how to analyze sensor band fairness in your distillation experiments.

## Overview
- Analyze performance differences across CGM sensor bands
- Evaluate if distillation affects sensor band fairness  
- Generate comprehensive visualizations and reports
- Provide recommendations for improvement

## Sensor Bands in OhioT1DM Dataset
- Different continuous glucose monitor (CGM) sensor configurations
- May affect glucose reading accuracy and prediction performance

## Setup and Imports

In [ ]:
# Import required libraries
import sys
sys.path.append('../')  # Add parent directory to path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import json

# Import our sensor fairness analyzer
from sensor_fairness_analyzer import SensorFairnessAnalyzer

# Set up visualization style
plt.style.use('default')
sns.set_palette("husl")

print("📡 Sensor Band Fairness Analysis System Loaded!")
print("🎯 Ready to analyze sensor band fairness in your diabetes prediction models")

# Initialize analyzer
analyzer = SensorFairnessAnalyzer()
print(f"📊 Patient data loaded: {len(analyzer.patient_data)} patients")

# Quick sensor band overview
sensor_counts = {}
for patient_id, sensor_band in analyzer.patient_data.items():
    sensor_counts[sensor_band] = sensor_counts.get(sensor_band, 0) + 1

print(f"📈 Sensor Band Distribution: {sensor_counts}")

## 1. Sensor Band Demographics Analysis

In [ ]:
# Load patient data from CSV to get full demographics
data_df = pd.read_csv("/workspace/LLM-TIME/data/ohiot1dm/data.csv")

print("📊 SENSOR BAND DEMOGRAPHICS")
print("=" * 50)

# Sensor band distribution
sensor_distribution = data_df['Sensor Band'].value_counts()
total_patients = len(data_df)

print(f"Total patients: {total_patients}")
print("Sensor Band Distribution:")
for sensor_band, count in sensor_distribution.items():
    percentage = (count / total_patients) * 100
    print(f"  {sensor_band}: {count} patients ({percentage:.1f}%)")

# Visualize sensor band distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Sensor band pie chart
colors = plt.cm.Set3(np.linspace(0, 1, len(sensor_distribution)))
sensor_distribution.plot(kind='pie', ax=ax1, autopct='%1.1f%%', colors=colors)
ax1.set_title('Sensor Band Distribution')
ax1.set_ylabel('')

# Sensor band by pump model
if 'Pump Model' in data_df.columns:
    sensor_pump = pd.crosstab(data_df['Sensor Band'], data_df['Pump Model'])
    sensor_pump.plot(kind='bar', ax=ax2, color=['lightblue', 'lightgreen'])
    ax2.set_title('Sensor Band Distribution by Pump Model')
    ax2.set_xlabel('Sensor Band')
    ax2.set_ylabel('Count')
    ax2.legend(title='Pump Model')
    ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

print(f"\n📋 Patient Data Preview:")
display(data_df.head())

## 2. Load Distillation Experiment Results

In [ ]:
# Run sensor band fairness analysis
print("🔍 RUNNING SENSOR BAND FAIRNESS ANALYSIS")
print("=" * 50)

try:
    # Analyze latest experiment
    sensor_groups = analyzer.analyze_latest()
    
    if sensor_groups:
        print("✅ Sensor band analysis complete!")
        
        # Show results summary
        print(f"\n📊 Results Summary:")
        for sensor_band, patients in sensor_groups.items():
            if patients:
                avg_mse = np.mean([p['mse'] for p in patients])
                avg_mae = np.mean([p['mae'] for p in patients])
                avg_rmse = np.mean([p['rmse'] for p in patients])
                
                print(f"\n📡 {sensor_band}:")
                print(f"   Patient count: {len(patients)}")
                print(f"   Average MSE: {avg_mse:.6f}")
                print(f"   Average MAE: {avg_mae:.6f}")
                print(f"   Average RMSE: {avg_rmse:.6f}")
    else:
        print("❌ No sensor band results found!")
        
except Exception as e:
    print(f"❌ Error during analysis: {e}")
    print("Make sure you have run some distillation experiments first!")

## 3. Sensor Band Fairness Metrics

In [ ]:
if 'sensor_groups' in locals() and sensor_groups:
    # Calculate comprehensive fairness metrics
    sensor_performance = {}
    
    print("⚖️ SENSOR BAND FAIRNESS ANALYSIS")
    print("=" * 50)
    
    # Calculate performance for each sensor band
    for sensor_band, patients in sensor_groups.items():
        if patients:
            mse_values = [p['mse'] for p in patients]
            mae_values = [p['mae'] for p in patients]
            rmse_values = [p['rmse'] for p in patients]
            
            sensor_performance[sensor_band] = {
                'count': len(patients),
                'avg_mse': np.mean(mse_values),
                'avg_mae': np.mean(mae_values),
                'avg_rmse': np.mean(rmse_values),
                'std_mse': np.std(mse_values),
                'std_mae': np.std(mae_values),
                'std_rmse': np.std(rmse_values),
                'patients': patients
            }
    
    # Calculate fairness ratios if we have multiple sensor bands
    if len(sensor_performance) >= 2:
        # Find best and worst performing sensor bands
        sensor_bands = list(sensor_performance.keys())
        sensor_rmse = [(band, perf['avg_rmse']) for band, perf in sensor_performance.items()]
        sensor_rmse.sort(key=lambda x: x[1])
        
        best_sensor = sensor_rmse[0]
        worst_sensor = sensor_rmse[-1]
        
        # Calculate fairness metrics
        rmse_ratio = worst_sensor[1] / best_sensor[1]
        rmse_diff = worst_sensor[1] - best_sensor[1]
        
        print(f"🎯 Fairness Analysis Across {len(sensor_bands)} Sensor Bands:")
        print(f"   Best performing: {best_sensor[0]} (RMSE: {best_sensor[1]:.6f})")
        print(f"   Worst performing: {worst_sensor[0]} (RMSE: {worst_sensor[1]:.6f})")
        print(f"   Performance Difference: {rmse_diff:.6f}")
        print(f"   Performance Ratio: {rmse_ratio:.2f}x")
        
        # Fairness assessment
        if rmse_ratio <= 1.1:
            fairness_level = "EXCELLENT"
            fairness_color = "🟢"
        elif rmse_ratio <= 1.2:
            fairness_level = "GOOD" 
            fairness_color = "🟡"
        elif rmse_ratio <= 1.5:
            fairness_level = "MODERATE"
            fairness_color = "🟠"
        else:
            fairness_level = "POOR"
            fairness_color = "🔴"
            
        print(f"\n{fairness_color} Fairness Assessment: {fairness_level}")
        
        # Store results for later use
        fairness_results = {
            'best_sensor': best_sensor[0],
            'worst_sensor': worst_sensor[0],
            'rmse_ratio': rmse_ratio,
            'rmse_diff': rmse_diff,
            'fairness_level': fairness_level
        }
    
    else:
        print("⚠️ Need at least 2 sensor bands for fairness comparison")
        fairness_results = None

## 4. Performance Visualization Dashboard

In [ ]:
if 'sensor_performance' in locals() and sensor_performance:
    # Create comprehensive visualization dashboard
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))
    
    # 1. Average Performance Comparison
    bands = list(sensor_performance.keys())
    rmse_values = [sensor_performance[band]['avg_rmse'] for band in bands]
    colors = plt.cm.Set3(np.linspace(0, 1, len(bands)))
    
    bars1 = ax1.bar(bands, rmse_values, color=colors, alpha=0.7)
    ax1.set_xlabel('Sensor Band')
    ax1.set_ylabel('Average RMSE')
    ax1.set_title('Average RMSE by Sensor Band')
    ax1.tick_params(axis='x', rotation=45)
    ax1.grid(True, alpha=0.3)
    
    # Add value labels on bars
    for bar, value in zip(bars1, rmse_values):
        ax1.text(bar.get_x() + bar.get_width()/2., bar.get_height() + bar.get_height()*0.01,
                f'{value:.4f}', ha='center', va='bottom', fontweight='bold', fontsize=10)
    
    # 2. Patient Count Distribution
    patient_counts = [sensor_performance[band]['count'] for band in bands]
    ax2.pie(patient_counts, labels=bands, autopct='%1.0f patients', 
            colors=colors, startangle=90)
    ax2.set_title('Patient Distribution by Sensor Band')
    
    # 3. Performance Distribution (Box Plot)
    rmse_data = []
    band_labels = []
    
    for band in bands:
        patients = sensor_performance[band]['patients']
        rmse_values = [p['rmse'] for p in patients]
        rmse_data.extend(rmse_values)
        band_labels.extend([band] * len(rmse_values))
    
    # Create DataFrame for seaborn
    plot_df = pd.DataFrame({'Sensor Band': band_labels, 'RMSE': rmse_data})
    sns.boxplot(data=plot_df, x='Sensor Band', y='RMSE', ax=ax3)
    ax3.set_title('RMSE Distribution by Sensor Band')
    ax3.tick_params(axis='x', rotation=45)
    ax3.grid(True, alpha=0.3)
    
    # 4. Fairness Summary
    ax4.text(0.1, 0.9, 'SENSOR BAND FAIRNESS SUMMARY', fontsize=14, fontweight='bold')
    
    if 'fairness_results' in locals() and fairness_results:
        summary_text = [
            f"Sensor Bands Analyzed: {len(bands)}",
            f"",
            f"Performance Ranking:",
            f"  Best: {fairness_results['best_sensor']}",
            f"  Worst: {fairness_results['worst_sensor']}",
            f"",
            f"Fairness Metrics:",
            f"  Performance Ratio: {fairness_results['rmse_ratio']:.2f}x",
            f"  Assessment: {fairness_results['fairness_level']}",
            f"",
            f"Total Patients: {sum(patient_counts)}"
        ]
    else:
        summary_text = [
            f"Sensor Bands Found: {len(bands)}",
            f"Total Patients: {sum(patient_counts)}",
            "",
            "Need multiple sensor bands",
            "for fairness comparison"
        ]
    
    for i, text in enumerate(summary_text):
        ax4.text(0.1, 0.8 - i*0.08, text, fontsize=11, transform=ax4.transAxes)
    
    ax4.set_xlim(0, 1)
    ax4.set_ylim(0, 1)
    ax4.axis('off')
    
    plt.tight_layout()
    plt.show()

## 5. Individual Patient Performance Analysis

In [ ]:
if 'sensor_groups' in locals() and sensor_groups:
    # Analyze individual patient performance
    all_patients = []
    
    for sensor_band, patients in sensor_groups.items():
        for patient in patients:
            all_patients.append({
                'Patient ID': patient['patient_id'],
                'Sensor Band': sensor_band,
                'MSE': patient['mse'],
                'MAE': patient['mae'],
                'RMSE': patient['rmse']
            })
    
    if all_patients:
        patient_df = pd.DataFrame(all_patients)
        
        print("👤 INDIVIDUAL PATIENT PERFORMANCE")
        print("=" * 50)
        display(patient_df)
        
        # Create individual performance visualizations
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))
        
        # 1. Scatter plot of individual performance
        for sensor_band in sensor_groups.keys():
            band_data = patient_df[patient_df['Sensor Band'] == sensor_band]
            ax1.scatter(band_data.index, band_data['RMSE'], 
                       label=sensor_band, alpha=0.7, s=100)
        
        ax1.set_xlabel('Patient Index')
        ax1.set_ylabel('RMSE')
        ax1.set_title('Individual Patient RMSE by Sensor Band')
        ax1.legend()
        ax1.grid(True, alpha=0.3)
        
        # 2. Performance comparison violin plot
        sns.violinplot(data=patient_df, x='Sensor Band', y='RMSE', ax=ax2)
        ax2.set_title('RMSE Distribution by Sensor Band')
        ax2.tick_params(axis='x', rotation=45)
        ax2.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Performance statistics
        print("\n📊 PERFORMANCE STATISTICS BY SENSOR BAND:")
        stats = patient_df.groupby('Sensor Band')['RMSE'].agg(['mean', 'std', 'min', 'max', 'count'])
        display(stats)

## 6. Clinical and Technical Insights

In [ ]:
if 'sensor_performance' in locals() and len(sensor_performance) >= 2:
    print("🔬 SENSOR BAND CLINICAL AND TECHNICAL INSIGHTS")
    print("=" * 60)
    
    # Sensor technology insights
    print("📡 SENSOR BAND ANALYSIS:")
    print("   • Sensor bands may represent different CGM configurations")
    print("   • Different sensor placements or calibration protocols")
    print("   • May affect glucose reading accuracy and prediction models")
    
    if 'fairness_results' in locals() and fairness_results:
        print(f"\n⚖️ FAIRNESS ASSESSMENT:")
        if fairness_results['rmse_ratio'] <= 1.1:
            print("✅ EXCELLENT sensor band fairness!")
            print("   • Models perform equally well across sensor configurations")
            print("   • No sensor-specific bias detected")
            print("   • Sensor differences don't affect prediction accuracy")
        elif fairness_results['rmse_ratio'] <= 1.2:
            print("👍 GOOD sensor band fairness with minor differences")
            print("   • Acceptable performance variation between sensor bands")
            print("   • Consider minor calibration adjustments")
        else:
            print("⚠️ SIGNIFICANT sensor band fairness issues detected!")
            print("   • Performance varies notably between sensor bands")
            print("   • May require sensor-band-specific model training")
        
        print(f"\n🎯 RECOMMENDATIONS:")
        
        if fairness_results['rmse_ratio'] <= 1.2:
            print("✅ Current approach is working well:")
            print("   • Continue unified model training across sensor bands")
            print("   • Monitor fairness in future experiments")
            print("   • Document sensor band fairness as model strength")
        else:
            print("🛠️ IMPROVEMENT ACTIONS NEEDED:")
            print("   • Investigate sensor band differences")
            print(f"   • Collect more training data for {fairness_results['worst_sensor']}")
            print("   • Consider sensor-band-specific feature engineering")
            print("   • Implement sensor-aware model architecture")
        
        print(f"\n💡 TECHNICAL CONSIDERATIONS:")
        print("   • Sensor bands may indicate different:")
        print("     - Calibration protocols")
        print("     - Sensor placement locations") 
        print("     - CGM device generations")
        print("     - Data collection periods")
        print("   • Consider sensor-band-specific normalization")
        print("   • Investigate temporal effects if bands represent time periods")

elif len(sensor_performance) == 1:
    print("📊 SINGLE SENSOR BAND ANALYSIS:")
    band_name = list(sensor_performance.keys())[0]
    print(f"   Only {band_name} patients in dataset")
    print("   Cannot assess sensor band fairness")
    print("   Consider investigating sensor band definitions")
else:
    print("⚠️ No sensor band performance data available")

## 7. Save Results and Generate Report

In [ ]:
# Save comprehensive results
if 'sensor_performance' in locals() and sensor_performance:
    print("💾 SAVING SENSOR BAND FAIRNESS RESULTS")
    print("=" * 50)
    
    # Prepare results for saving
    results_summary = {
        'analysis_type': 'sensor_band_fairness',
        'timestamp': pd.Timestamp.now().isoformat(),
        'sensor_bands_analyzed': list(sensor_performance.keys()),
        'total_patients': sum([perf['count'] for perf in sensor_performance.values()]),
        'sensor_performance': sensor_performance
    }
    
    # Add fairness metrics if available
    if 'fairness_results' in locals() and fairness_results:
        results_summary['fairness_metrics'] = fairness_results
    
    # Convert numpy types for JSON serialization
    def convert_numpy(obj):
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        elif isinstance(obj, np.number):
            return float(obj)
        elif isinstance(obj, dict):
            return {key: convert_numpy(value) for key, value in obj.items()}
        elif isinstance(obj, list):
            return [convert_numpy(item) for item in obj]
        return obj
    
    results_summary = convert_numpy(results_summary)
    
    # Save to file
    output_file = "sensor_band_fairness_results.json"
    with open(output_file, 'w') as f:
        json.dump(results_summary, f, indent=2)
    
    print(f"✅ Results saved to: {output_file}")
    
    # Display final summary
    print(f"\n📋 FINAL SUMMARY:")
    print(f"   Sensor bands analyzed: {len(sensor_performance)}")
    print(f"   Total patients: {results_summary['total_patients']}")
    if 'fairness_metrics' in results_summary:
        print(f"   Fairness ratio: {results_summary['fairness_metrics']['rmse_ratio']:.2f}x")
        print(f"   Fairness level: {results_summary['fairness_metrics']['fairness_level']}")
    
else:
    print("⚠️ No results to save - run the analysis first!")

## 8. Next Steps and Integration

### If Sensor Band Fairness is Excellent (≤1.2x ratio):
- ✅ **Great job!** Your model is sensor-band-fair
- 📊 Continue monitoring fairness in future experiments  
- 🔄 Use this as a baseline for model improvements
- 📝 Document this fairness strength in your research

### If Sensor Band Fairness Needs Improvement (>1.2x ratio):
- 🔍 **Investigation**: Understand what sensor bands represent
- 📚 **Data Collection**: Gather more data for underperforming sensor bands
- 🛠️ **Feature Engineering**: Create sensor-band-specific features
- ⚖️ **Fairness Constraints**: Implement fairness training (see `Fairness_Integration_Guide.ipynb`)

### Additional Analyses:
- 👫 **Gender Fairness**: Run `Gender_Fairness_Analysis.ipynb`
- 🎂 **Age Fairness**: Check `Age_Group_Fairness_Analysis.ipynb`  
- 📱 **Pump Model Fairness**: Analyze `Pump_Model_Fairness_Analysis.ipynb`
- 🏆 **Comprehensive**: Use `Legendary_Fairness_Analysis.ipynb` for all features

### Technical Integration:
- 🔧 See `fairness/integration_guide.py` for implementation
- 📖 Review fairness loss functions and training modifications
- 🧪 Experiment with sensor-band stratified training approaches
- 🔬 Investigate sensor band definitions and clinical implications